In [20]:
import pandas as pd
from rpy2.robjects import r
import rpy2.robjects.pandas2ri as pandas2ri

#load into the env
file="~/govWebsites/rfiles/d.Rdata"
rf=r['load'](file)

'd'


In [29]:
d = r['d']

In [31]:
d['doc']

1        was mayor shepherd city clerk and board member...
2        those present were dale grant craft mayor shep...
3        was mayor shepherd city clerk and board member...
4        those present were dale grant craft mayor shep...
5        notice on at pm in the city council room prese...
6        were mayor shepherd city clerk and board membe...
7        those present were dale grant craft mayor shep...
8        those present were grant craft made a motion t...
9        were mayor shepherd city clerk and board membe...
10       notice on at pm in the city council room prese...
11       pm those present were grant craft made a motio...
12       notice on at pm in the city council room prese...
13       notice on at pm in the city council room prese...
14       those present were grant craft made a motion t...
15       present were board members and huckleberry cit...
16       pm those present were grant mayor shepherd and...
17       opened the public hearing for the proposed cum.

In [33]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(d['doc'])
X_train_counts.shape

(16116, 35257)

In [35]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(16116, 35257)

Naive Bayes

In [36]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, d['winner'])

In [77]:
docs_new = ['God is love', 'OpenGL on the GPU is fast', 'we like spending lots of money on social issues', 'waste management', 'climate change']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

In [78]:
predicted

array(['Republican', 'Democratic', 'Democratic', 'Democratic', 'Democratic'], 
      dtype='<U10')

In [43]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB()),
 ])

In [45]:
text_clf.fit(d['doc'], d['winner'])

Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [46]:
from sklearn.model_selection import train_test_split

In [47]:
X_train, X_test, y_train, y_test = train_test_split(d['doc'], d['winner'], test_size=0.33, random_state=42)

In [50]:
text_clf.fit(X_train,y_train)

Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [51]:
import numpy as np
predicted = text_clf.predict(X_test)
np.mean(predicted == y_test)        

0.73058845647678139

SVM:

In [68]:
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                           alpha=0.001, random_state=42)),
])
text_clf.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...     penalty='l2', power_t=0.5, random_state=42, shuffle=True, verbose=0,
       warm_start=False))])

In [69]:
predicted = text_clf.predict(X_test)
np.mean(predicted == y_test)

0.79789434104154922

Grid search parameters

In [56]:
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
              'tfidf__use_idf': (True, False),
              'clf__alpha': (1e-2, 1e-3),
}

In [57]:
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)

In [61]:
gs_clf = gs_clf.fit(X_train[:4000],y_train[:4000])

In [62]:
gs_clf.best_score_

0.77400000000000002

In [63]:
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

clf__alpha: 0.001
tfidf__use_idf: True
vect__ngram_range: (1, 1)
